# Neural networks

### Zhentao Shi

* Neural network (ANN) is the workhorse of AI
* A type of nonlinear models (with a structure)

![NN](graph/Colored_neural_network.png)

## Layers

* The transition from layer $k-1$ to layer $k$ can be written as

$$
\begin{align*}
z_l^{(k)} & = w_{l0}^{(k-1)} + \sum_{j=1}^{p_{k-1} } w_{lj}^{(k-1)} a_j^{(k-1)} \\ 
a_l^{(k)} & = g^{(k)} ( z_l^{(k)})
\end{align*}
$$

where $a_j^{(0)} = x_j$ is the input.

* The latent variable $z_l^{(k)}$ usually takes a linear form
* *Activation function* $g(\cdot)$ is usually a simple nonlinear function
* Popular choice: sigmoid ($1/(1+\exp(-x))$); ReLu, $z\cdot 1\{x\geq 0\}$

## Hyperparameters

A user has several decisions to make
* Activation function
* Number of hidden layers
* Number of nodes in each layer


* Many free parameters are generated from the multiple layer and multiple nodes
* Regularization methods can be employed to penalize
the $l_1$ and/or $l_2$ norms, which requires extra tuning parameters

## Why Does It Work?

* Animated video by [3Blue1Brown](https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi)

#  Theory is Incomplete

* Theoretical understanding is an ongoing endeavor.
* Hornik, Stinchcombe, and White (1989):
    * A single hidden layer neural network, given enough many nodes, is a *universal approximator* for any
measurable function.

# Optimization

* Free parameters must be determined by numerical optimization
* Nonlinear multiplicative structure makes the optimization challenging
* Secure the global optimizer is almost impossible
* De facto optimization algorithm is *stochastic gradient descent*



* Google's `tensorflow`
* `keras` is the deep learning modeling language